In [1]:
# Load Libraries
library(Seurat)
library(SeuratWrappers)
library(ggplot2)

# Set Working Directory
setwd("../")

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t




# WT Organoid

## Initial Processing

In [ ]:
path.wto <- "GBMProject/XeniumData/Organoid_WT"

# Load the Xenium data
xenium.wto <- LoadXenium(path.wto, fov = "fov")

# remove cells with 0 counts
xenium.wto <- subset(xenium.wto, subset = nCount_Xenium > 0)

xenium.wto

In [ ]:
options(repr.plot.width=5, repr.plot.height=5) # To set the figure size in Jupyter
VlnPlot(xenium.wto, features = c("nFeature_Xenium", "nCount_Xenium"), ncol = 2, pt.size = 0)

In [ ]:
xenium.wto <- NormalizeData(xenium.wto)
xenium.wto <- FindVariableFeatures(xenium.wto, selection.method = "vst")
xenium.wto <- ScaleData(xenium.wto, features = rownames(xenium.wto))
xenium.wto <- RunPCA(xenium.wto, npcs = 30, features = rownames(xenium.wto), verbose = FALSE)
xenium.wto <- RunUMAP(xenium.wto, dims = 1:15, verbose = FALSE)
xenium.wto <- FindNeighbors(xenium.wto, reduction = "pca", dims = 1:15, verbose = FALSE)
xenium.wto <- FindClusters(xenium.wto, resolution = 0.2, verbose = FALSE)

## Cell Type Annotation

In [ ]:
scRNA.data <- readRDS("GBMProject/GBMOrganoid/Checkpoints/1_DataPreprocessing.rds")
counts_matrix <- scRNA.data[["RNA"]]$data
write.csv(counts_matrix, file='GBMProject/GBMOrganoid/scrna_mtx.csv', quote=F, row.names=T)
Idents(scRNA.data) <- 'celltype2'

In [ ]:
anchors <- FindTransferAnchors(reference = scRNA.data, query = xenium.wto, normalization.method = "LogNormalize")
anchors

In [ ]:
predictions <- TransferData(anchorset = anchors, refdata = scRNA.data$celltype, prediction.assay = FALSE,
    weight.reduction = xenium.wto[["pca"]], dims = 1:30)

xenium.wto$predicted.celltype <- predictions$predicted.id

In [ ]:
predictions2 <- TransferData(anchorset = anchors, refdata = scRNA.data$celltype2, prediction.assay = FALSE,
    weight.reduction = xenium.wto[["pca"]], dims = 1:30)

xenium.wto$predicted.celltype2 <- predictions2$predicted.id

In [ ]:
table(xenium.wto$predicted.celltype)

In [ ]:
options(repr.plot.width=15, repr.plot.height=10) # To set the figure size in Jupyter

celltype.plot <- ImageDimPlot(xenium.wto, group.by = "predicted.celltype2", size = 1.5, dark.background = TRUE,
                             cols=c("Mesenchyme-1"="mediumpurple1","RGC-2"="skyblue2","Proliferating"="dimgray","Neuronal"="navyblue","RGC-1"="skyblue1",
              "Proliferating"="dimgray","RGC-3"="skyblue3","Neuroepithelial"="lightcoral","Mesenchyme-2"="mediumpurple3",
              "RGC-1"="skyblue1","NeuralCrest"="lightseagreen","Retinal"="goldenrod1","RGC-2"="skyblue2",
              "RGC-3"="skyblue3","Endothelial"="gray","Microglia"="lightgreen")) + ggtitle("Cell type")

celltype.plot

## Save Data

In [ ]:
# write filtered barcodes 
write.csv(colnames(xenium.wto), file='GBMProject/GBMOrganoid/Barcodes_WT.csv', quote=F, row.names=T)

# write predicted cell tpyes
write.csv(xenium.wto$predicted.celltype, file='GBMProject/GBMOrganoid/PredCellType_WT.csv', quote=F, row.names=T)

In [ ]:
saveRDS(xenium.wto, file = "GBMProject/GBMOrganoid/Checkpoints/XENIUM_WT.rds")

# PRO Organoid

## Initial Processing

In [ ]:
path.pro <- "GBMProject/XeniumData/Organoid_PRO"
# Load the Xenium data
xenium.pro <- LoadXenium(path.pro, fov = "fov")
# remove cells with 0 counts
xenium.pro <- subset(xenium.pro, subset = nCount_Xenium > 0)

In [ ]:
options(repr.plot.width=5, repr.plot.height=5) # To set the figure size in Jupyter
VlnPlot(xenium.pro, features = c("nFeature_Xenium", "nCount_Xenium"), ncol = 2, pt.size = 0)

In [ ]:
xenium.pro <- NormalizeData(xenium.pro)
xenium.pro <- FindVariableFeatures(xenium.pro, selection.method = "vst")
xenium.pro <- ScaleData(xenium.pro, features = rownames(xenium.pro))
xenium.pro <- RunPCA(xenium.pro, npcs = 30, features = rownames(xenium.pro), verbose = FALSE)
xenium.pro <- RunUMAP(xenium.pro, dims = 1:15, verbose = FALSE)
xenium.pro <- FindNeighbors(xenium.pro, reduction = "pca", dims = 1:15, verbose = FALSE)
xenium.pro <- FindClusters(xenium.pro, resolution = 0.2, verbose = FALSE)

## Cell Type Annotation

In [ ]:
anchors <- FindTransferAnchors(reference = scRNA.data, query = xenium.pro, normalization.method = "LogNormalize")
anchors

In [ ]:
predictions <- TransferData(anchorset = anchors, refdata = scRNA.data$celltype, prediction.assay = FALSE,
    weight.reduction = xenium.pro[["pca"]], dims = 1:30)

xenium.pro$predicted.celltype <- predictions$predicted.id

In [ ]:
predictions2 <- TransferData(anchorset = anchors, refdata = scRNA.data$celltype2, prediction.assay = FALSE,
    weight.reduction = xenium.pro[["pca"]], dims = 1:30)

xenium.pro$predicted.celltype2 <- predictions2$predicted.id

In [ ]:
table(xenium.pro$predicted.celltype)

In [ ]:
options(repr.plot.width=15, repr.plot.height=10) # To set the figure size in Jupyter

celltype.plot <- ImageDimPlot(xenium.pro, group.by = "predicted.celltype2", size = 1.5, dark.background = F,
                             cols=c("Mesenchyme-1"="mediumpurple1","RGC-2"="skyblue2","Proliferating"="dimgray","Neuronal"="navyblue","RGC-1"="skyblue1",
              "Proliferating"="dimgray","RGC-3"="skyblue3","Neuroepithelial"="lightcoral","Mesenchyme-2"="mediumpurple3",
              "RGC-1"="skyblue1","NeuralCrest"="lightseagreen","Retinal"="goldenrod1","RGC-2"="skyblue2",
              "RGC-3"="skyblue3","Endothelial"="gray","Microglia"="lightgreen")) + ggtitle("Cell type")

celltype.plot

## Save Data

In [ ]:
# write filtered barcodes 
write.csv(colnames(xenium.pro), file='GBMProject/GBMOrganoid/Barcodes_PRO.csv', quote=F, row.names=T)

# write predicted cell tpyes
write.csv(xenium.pro$predicted.celltype, file='GBMProject/GBMOrganoid/PredCellType_PRO.csv', quote=F, row.names=T)

In [ ]:
saveRDS(xenium.pro, file = "GBMProject/GBMOrganoid/Checkpoints/XENIUM_PRO.rds")

# MES Organoid

## Iniital Processing

In [ ]:
path.mes <- "GBMProject/XeniumData/Organoid_MES"
# Load the Xenium data
xenium.mes <- LoadXenium(path.mes, fov = "fov")
# remove cells with 0 counts
xenium.mes <- subset(xenium.mes, subset = nCount_Xenium > 0)

In [ ]:
options(repr.plot.width=5, repr.plot.height=5) # To set the figure size in Jupyter
VlnPlot(xenium.mes, features = c("nFeature_Xenium", "nCount_Xenium"), ncol = 2, pt.size = 0)

In [ ]:
xenium.mes <- NormalizeData(xenium.mes)
xenium.mes <- FindVariableFeatures(xenium.mes, selection.method = "vst")
xenium.mes <- ScaleData(xenium.mes, features = rownames(xenium.mes))
xenium.mes <- RunPCA(xenium.mes, npcs = 30, features = rownames(xenium.mes), verbose = FALSE)
xenium.mes <- RunUMAP(xenium.mes, dims = 1:15, verbose = FALSE)
xenium.mes <- FindNeighbors(xenium.mes, reduction = "pca", dims = 1:15, verbose = FALSE)
xenium.mes <- FindClusters(xenium.mes, resolution = 0.2, verbose = FALSE)

In [ ]:
xenium.mes <- FindSpatiallyVariableFeatures(xenium.mes, assay = "SCT", features = VariableFeatures(xenium.mes),
    selection.method = "moransi")

## Cell Type Annotation

In [ ]:
anchors <- FindTransferAnchors(reference = scRNA.data, query = xenium.mes, normalization.method = "LogNormalize")
anchors

In [ ]:
predictions <- TransferData(anchorset = anchors, refdata = scRNA.data$celltype, prediction.assay = FALSE,
    weight.reduction = xenium.mes[["pca"]], dims = 1:30)

xenium.mes$predicted.celltype <- predictions$predicted.id

In [ ]:
predictions2 <- TransferData(anchorset = anchors, refdata = scRNA.data$celltype2, prediction.assay = FALSE,
    weight.reduction = xenium.mes[["pca"]], dims = 1:30)

xenium.mes$predicted.celltype2 <- predictions2$predicted.id

In [ ]:
table(xenium.mes$predicted.celltype)

In [ ]:
options(repr.plot.width=15, repr.plot.height=10) # To set the figure size in Jupyter

celltype.plot <- ImageDimPlot(xenium.mes, group.by = "predicted.celltype2", size = 1.5, dark.background = F,
                             cols=c("Mesenchyme-1"="mediumpurple1","RGC-2"="skyblue2","Proliferating"="dimgray","Neuronal"="navyblue","RGC-1"="skyblue1",
              "Proliferating"="dimgray","RGC-3"="skyblue3","Neuroepithelial"="lightcoral","Mesenchyme-2"="mediumpurple3",
              "RGC-1"="skyblue1","NeuralCrest"="lightseagreen","Retinal"="goldenrod1","RGC-2"="skyblue2",
              "RGC-3"="skyblue3","Endothelial"="gray","Microglia"="lightgreen")) + ggtitle("Cell type")

celltype.plot

## Save Data

In [ ]:
# write filtered barcodes 
write.csv(colnames(xenium.mes), file='GBMProject/GBMOrganoid/Barcodes_MES.csv', quote=F, row.names=T)

# write predicted cell tpyes
write.csv(xenium.mes$predicted.celltype, file='GBMProject/GBMOrganoid/PredCellType_MES.csv', quote=F, row.names=T)

In [ ]:
saveRDS(xenium.mes, file = "GBMProject/GBMOrganoid/Checkpoints/XENIUM_MES.rds")

# Cell Type Proportions

In [ ]:
# Calculate cell type proportions
prop.celltype.wt <- data.frame(prop.table(table(xenium.wto$predicted.celltype)))
prop.celltype.wt$HTO <- 'WT'
prop.celltype.pro <- data.frame(prop.table(table(xenium.pro$predicted.celltype)))
prop.celltype.pro$HTO <- 'PRO'
prop.celltype.mes <- data.frame(prop.table(table(xenium.mes$predicted.celltype)))
prop.celltype.mes$HTO <- 'MES'

prop.celltype <- rbind(prop.celltype.wt, prop.celltype.pro)
prop.celltype <- rbind(prop.celltype, prop.celltype.mes)

prop.celltype